In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [2]:
sf_data = pd.read_csv("./raw_data/sanfrancisco.csv")

In [3]:
raw_columns = list(sf_data)

In [4]:
unused_columns = ['snow_1h', 'snow_24h', 'rain_24h', 'rain_1h', 'snow_3h', 'rain_today', 'snow_today', 'weather_icon', 'weather_id', 'sea_level', 'grnd_level', 'lat', 'lon', 'city_id', 'city_name']

In [5]:
used_columns = columns = list(set(raw_columns) - set(unused_columns))

In [6]:
sf_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38415 entries, 0 to 38414
Data columns (total 28 columns):
dt                     38415 non-null int64
dt_iso                 38415 non-null object
city_id                38415 non-null int64
city_name              0 non-null float64
lat                    0 non-null float64
lon                    0 non-null float64
temp                   38415 non-null float64
temp_min               38415 non-null float64
temp_max               38415 non-null float64
pressure               38415 non-null int64
sea_level              0 non-null float64
grnd_level             0 non-null float64
humidity               38415 non-null int64
wind_speed             38415 non-null int64
wind_deg               38415 non-null int64
rain_1h                1866 non-null float64
rain_3h                1355 non-null float64
rain_24h               97 non-null float64
rain_today             137 non-null float64
snow_1h                3 non-null float64
snow_3h        

In [7]:
def cleanup(raw_data, used_columns):
    data = raw_data.loc[:, used_columns] 
    print("fill_na")
    data['rain_3h'] = data['rain_3h'].fillna(0)
    print("drop_duplicates")
    data.drop_duplicates('dt', inplace=True)
    print("add_new_dada")
    data = add_new_data(data)
    
    data = data.apply(transform_datetime, axis=1)
    
    unused_columns  = ['dt_iso', 'weather_main', 'weather_description', 'dt_datetime']
    
    data = data.drop(unused_columns, axis=1)
    data = data.reset_index(drop=True)
    return data

def add_new_data(data):
    data['dt_datetime'] =  pd.to_datetime(data['dt_iso'], format='%Y-%m-%d %H:%M:%S +%f %Z')
    weather_description_columns = list(set(data['weather_description']))
    weather_main_columns = list(set(data['weather_main']))
    data = transform_categorical_data(data, weather_description_columns,weather_main_columns)

    return data

def transform_categorical_data(data, weather_description_columns, weather_main_columns):
    for column in weather_description_columns:
        data[column] = data['weather_description'] == column
        data[column] = data[column].astype(int)
        
    for column in weather_main_columns:
        data[column] = data['weather_main'] == column
        data[column] = data[column].astype(int)
    return data
    
def transform_datetime(current_data):
    for month in range(1, 12):
        current_data['month_{}'.format(month)] = 1 if current_data['dt_datetime'].month == month else 0

    current_data['year'] =  current_data['dt_datetime'].year
    current_data['dayofweek'] = current_data['dt_datetime'].dayofweek
    current_data['dayofyear'] = current_data['dt_datetime'].dayofyear
    current_data['hourofday'] = current_data['dt_datetime'].hour
    return current_data


In [8]:
sf_data2 = cleanup(sf_data, used_columns)


fill_na
drop_duplicates
add_new_dada


In [9]:
def add_target_data(data):
    data['target_temp'] = data['temp'][1:].append(pd.Series([np.nan]) , ignore_index=True)
    return data

In [10]:
sf_data3 = add_target_data(sf_data2)

In [11]:
# def add_previous_datum(raw_data):
#     data = raw_data.copy()
#     diff_columns = list(set(raw_data.columns) - set(['dt', 'dt_iso', 'dt_datetime']))
#     maximum_prev = 2 *  24
#     data = data.apply(add_previous_data, args=(diff_columns, data, maximum_prev), axis=1)
#     return data

# def add_previous_data(current_data,  diff_columns, raw_data, maximum_prev):
#     index = current_data.name
#     print(index)
#     if index == 0:
#         return current_data
#     prev_right = index - 1
#     prev_left = index - maximum_prev if index - maximum_prev > 0 else 0
       
#     while prev_left <= prev_right:
#         current_data = add_diff_data(current_data, raw_data.iloc[prev_left], maximum_prev, diff_columns)
#         prev_left += 1
#     return current_data

# def add_diff_data(current_data, prev_data, maximum_prev, diff_columns):
#     diff = int(pd.Timedelta(current_data['dt_datetime'] - prev_data['dt_datetime']).seconds/ 3600)
#     if diff > 0 and diff < maximum_prev:
#         for diff_column in diff_columns:
#             column_name = '{}_{}_ago'.format(diff_column, diff)
#             current_data[column_name] = prev_data[diff_column]
#     return current_data

# # add new data by merging np array and adding dummy data 
# #[NALL, NALL, data1, data2]


In [12]:
def add_diff_data(raw_data):
    data = raw_data.copy()
    diff_columns = list(set(raw_data.columns) - set(['dt', 'dt_iso', 'dt_datetime', 'target_temp', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'year', 'dayofyear', 'dayofweek', 'hourofday']))
    maximum_prev = 2 *  24
    for i in [1,2,3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48]:
        for column in diff_columns:
            data['{}_{}_ago'.format(column, i)] = pd.Series(np.repeat(np.nan, i)).append(data[column][:-i] , ignore_index=True)
    return data    

In [13]:
sf_data4 = add_diff_data(sf_data3)

In [14]:
sf_data4['target_temp']

0        289.13
1        290.73
2        293.02
3        296.18
4        299.66
5        300.03
6        301.07
7        302.29
8        304.70
9        304.86
10       304.08
11       302.69
12       300.52
13       298.56
14       297.94
15       296.95
16       294.14
17       293.46
18       293.32
19       292.39
20       291.96
21       291.52
22       290.75
23       290.26
24       297.49
25       299.48
26       301.24
27       302.04
28       303.25
29       302.50
          ...  
33412    285.81
33413    287.51
33414    288.89
33415    290.30
33416    290.57
33417    289.16
33418    286.82
33419    285.23
33420    283.67
33421    282.17
33422    281.00
33423    280.16
33424    279.89
33425    279.64
33426    278.60
33427    278.16
33428    277.70
33429    277.16
33430    276.52
33431    276.25
33432    276.50
33433    277.16
33434    281.23
33435    283.96
33436    286.48
33437    287.98
33438    289.79
33439    290.53
33440    290.15
33441       NaN
Name: target_temp, dtype